# 🧠 Hull Tactical Market Prediction — AutoGluon Baseline

This notebook builds a baseline model for the [**Hull Tactical Market Prediction**](https://www.kaggle.com/competitions/hull-tactical-market-prediction) competition using **AutoGluon Tabular**. The goal is to predict trading positions that maximize a Sharpe-like performance metric.  

## Overview
- **Task:** Predict next-period trading positions (long / flat) using engineered financial features.
- **Approach:** Train an AutoGluon model on historical data to predict *forward returns*, then post-process those predictions into positions for scoring and submission.
- **Metric:** Custom approximation of the competition’s adjusted Sharpe ratio, which penalizes volatility and underperformance.
- **Post-processing:** A unified `post_process_signal()` function ensures parity between local validation and leaderboard logic by converting model predictions into bounded investment positions.

---

In [1]:
from pathlib import Path
WHEELS = Path("/kaggle/input/autogluon-1-4-0-offline")  # <- your dataset

!pip install --no-index --quiet --find-links="{WHEELS}" \
  "torch==2.5.1" "torchvision==0.20.1" "torchaudio==2.5.1" "bitsandbytes>=0.46.1" "mlforecast==0.14.0" "optuna==4.3.0"

!pip install --no-index --quiet --find-links="{WHEELS}" \
    "autogluon.tabular"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from autogluon.tabular import TabularPredictor

# === Metric constants (define to avoid NameError in custom scorer) ===
ALPHA_FOR_SCORER = 0.600132
TAU_ABS_FOR_SCORER = 9.43717e-05
MIN_INVESTMENT, MAX_INVESTMENT = 0.0, 2.0
TRADING_DAYS = 252


# === Single source of truth for post-processing predictions ===
# Use this both in the custom scorer and at inference to ensure parity with leaderboard logic.
# Current behavior: long-only; open a position of size ALPHA when prediction > TAU; else 0; then clip to [MIN, MAX].
# If you later want symmetric long/short, add a flag and branch here.

def post_process_signal(y_pred,
                        *,
                        tau: float = TAU_ABS_FOR_SCORER,
                        alpha: float = ALPHA_FOR_SCORER,
                        min_investment: float = MIN_INVESTMENT,
                        max_investment: float = MAX_INVESTMENT):
    sig = np.asarray(y_pred, dtype=float).ravel()
    pos = np.where(sig > tau, alpha, 0.0)
    return np.clip(pos, min_investment, max_investment)


DATA_PATH="/kaggle/input/hull-tactical-market-prediction/"

train = pd.read_csv(f"{DATA_PATH}train.csv")

TARGET = "forward_returns"
# simple guard to surface a clear error if the label is missing
if TARGET not in train.columns:
    raise ValueError(f"Expected target column '{TARGET}' in train.csv; found: {list(train.columns)}")

DROP_IF_EXISTS = ["row_id", "id", "risk_free_rate", "market_forward_excess_returns"]
use_cols = [c for c in train.columns if c not in DROP_IF_EXISTS]
train = train[use_cols]


def predict(test: pl.DataFrame) -> float:
    """Return a single post-processed position for a **single-row** Polars DataFrame.

    This mirrors the scorer's post-processing to keep CV ↔ leaderboard parity.
    - Drops leak-prone / non-feature columns if present.
    - Aligns columns to the predictor's feature set.
    - Converts Polars → Pandas for AutoGluon.
    """
    if not isinstance(test, pl.DataFrame):
        raise TypeError("predict(test): expected a Polars DataFrame input")

    if test.height != 1:
        raise ValueError(f"predict(test): expected a single-row Polars DataFrame, got {test.height} rows")

    # Drop known non-feature columns if present
    drop_cols = [c for c in DROP_IF_EXISTS if c in test.columns]
    test_pl = test.drop(drop_cols) if drop_cols else test

    # Ensure target isn't present at inference
    if TARGET in test_pl.columns:
        test_pl = test_pl.drop(TARGET)

    # Convert to pandas for AutoGluon
    test_pd = test_pl.to_pandas()

    # Align to the model's feature set (adds missing as 0, drops extras)
    feats = predictor.feature_metadata.get_features() if 'predictor' in globals() else [c for c in train.columns if c != TARGET]
    #test_pd = test_pd.reindex(columns=feats, fill_value=0)

    # Raw prediction → post-processed trading position
    raw = predictor.predict(test_pd)
    pos = post_process_signal(raw)
    return float(np.asarray(pos).ravel()[0])




In [3]:
predictor = TabularPredictor(
    label=TARGET,
    eval_metric="rmse",
    problem_type="regression"  # predicting returns is typically regression
)

predictor.fit(
    train_data=train,
    presets="high_quality",  # good speed/quality tradeoff to start
    time_limit= 60 * 60 *9
)

No path specified. Models will be saved in: "AutogluonModels/ag-20251212_180948"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Sep 27 10:16:09 UTC 2025
CPU Count:          4
Memory Avail:       30.24 GB / 31.35 GB (96.4%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['high_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setti

[1000]	valid_set's rmse: 0.0102018
[2000]	valid_set's rmse: 0.0101279
[3000]	valid_set's rmse: 0.0101045


	-0.0105	 = Validation score   (-root_mean_squared_error)
	45.14s	 = Training   runtime
	0.28s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 2652.49s of the 2652.38s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)


[1000]	valid_set's rmse: 0.0103357
[2000]	valid_set's rmse: 0.0103083
[3000]	valid_set's rmse: 0.0103035
[1000]	valid_set's rmse: 0.0108771
[2000]	valid_set's rmse: 0.0108357


	-0.0104	 = Validation score   (-root_mean_squared_error)
	83.52s	 = Training   runtime
	0.46s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 2568.38s of the 2568.27s of remaining time.
	-0.0106	 = Validation score   (-root_mean_squared_error)
	340.06s	 = Training   runtime
	0.97s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ... Training model for up to 2226.77s of the 2226.66s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
	-0.0105	 = Validation score   (-root_mean_squared_error)
	49.81s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L2 ... Training model for up to 2176.77s of the 2176.66s of remaining time.
	-0.0107	 = Validation score   (-root_mean_squared_error)
	35.0s	 = Training   runtime
	0.97s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L2 ... Training model for up to 2140.15s of the 2140.04s

...

	-0.0104	 = Validation score   (-root_mean_squared_error)
	277.66s	 = Training   runtime
	3.52s	 = Validation runtime
Fitting model: NeuralNetFastAI_r191_BAG_L2 ... Training model for up to 1290.57s of the 1290.47s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
No improvement since epoch 2: early stopping
No improvement since epoch 12: early stopping
No improvement since epoch 3: early stopping
No improvement since epoch 0: early stopping
No improvement since epoch 10: early stopping
No improvement since epoch 22: early stopping
	-0.0108	 = Validation score   (-root_mean_squared_error)
	165.4s	 = Training   runtime
	0.53s	 = Validation runtime
Fitting model: CatBoost_r9_BAG_L2 ... Training model for up to 1124.52s of the 1124.42s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
	Ran out of time, early stopping on

[1000]	valid_set's rmse: 0.0102776


	-0.0105	 = Validation score   (-root_mean_squared_error)
	17.45s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: NeuralNetTorch_r22_BAG_L2 ... Training model for up to 521.92s of the 521.81s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
	-0.0105	 = Validation score   (-root_mean_squared_error)
	178.97s	 = Training   runtime
	0.42s	 = Validation runtime
Fitting model: XGBoost_r33_BAG_L2 ... Training model for up to 342.38s of the 342.27s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
	-0.0105	 = Validation score   (-root_mean_squared_error)
	285.96s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: ExtraTrees_r42_BAG_L2 ... Training model for up to 56.08s of the 55.97s of remaining time.
	-0.0106	 = Validation score   (-root_mean_squared_error)
	26.44s	 = Training   runtime
	0

[1000]	valid_set's rmse: 0.0106064
[2000]	valid_set's rmse: 0.0106043


	-0.0105	 = Validation score   (-root_mean_squared_error)
	16.33s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 22434.55s of the 22434.54s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
	-0.0105	 = Validation score   (-root_mean_squared_error)
	160.49s	 = Training   runtime
	0.34s	 = Validation runtime
Fitting model: XGBoost_r33_BAG_L1 ... Training model for up to 22273.60s of the 22273.59s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
	-0.0105	 = Validation score   (-root_mean_squared_error)
	185.69s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTrees_r42_BAG_L1 ... Training model for up to 22087.72s of the 22087.70s of remaining time.
	-0.0109	 = Validation score   (-root_mean_squared_error)
	22.61s	 = Training

In [4]:
import kaggle_evaluation.default_inference_server as kis
import os

# ---------- KAGGLE SERVER BOOTSTRAP ----------
inference_server = kis.DefaultInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))